In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
   AutoModel,
)


tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)


In [4]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [5]:
import pandas as pd

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
#df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/movie/basic4_data.csv")
df_train=pd.read_csv("series8_data_replace2.csv")
#df_train=pd.read_csv("basic8_data_augment.csv") #testing
df_val=pd.read_csv("series8_val.csv")
df_total=pd.read_csv("series8_total.csv")
#df_val_replace=pd.read_csv("basic8_val_replace.csv")
len_=len(df_train)
series=["西方極樂園","絕命律師","怪奇物語","絕命毒師","獵魔士","黑鏡","魷魚遊戲","黑袍糾察隊"]
print("data len=",len_)
print("val len=",len(df_val))

data len= 3881
val len= 5189


In [8]:
#-------把太長的去掉------
MAX_LENGTH=80

df_train = df_train[~(df_train.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_val = df_val[~(df_val.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
#df_val_replace=df_val_replace[~(df_val_replace.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
len_=len(df_train)
print("after len=",len_)
print("after len=",len(df_val))

after len= 2719
after len= 4874


C:\Users\user\AppData\Local\Temp\ipykernel_24504\3786618029.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]


In [9]:
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用

print("訓練樣本數：", len(df_train))
print(df_train.head())
print(df_val.head())

訓練樣本數： 2719
                         sentence  label
3                       就看有沒有第五季了  西方極樂園
4  忍不住浮出來討論QQ但就千頭萬緒，沒辦法整理成有頭有尾的一篇  西方極樂園
5           前面幾季有著中央電腦在主導著人類社會的存在  西方極樂園
6        直到德妹從人工園區進入人類社會並將中央電腦給解決  西方極樂園
7        這一季德妹後來才知道自己成為了中央電腦的一段程式  西方極樂園
                                            sentence  label
1                                      但是這世界下，各種暗流湧動  西方極樂園
2          部分機器人出現自我覺醒，發現了自己只是作為故事角色的存在，並且想擺脫樂園對其的控制  西方極樂園
3  樂園的管理層害怕樂園的創造者控制著樂園的一切而試圖奪取其控制權，但樂園創造者則不會善罷甘休，...  西方極樂園
4                       而買下樂園的一名神秘男子試圖重新發現當年旅程所留下的謎團  西方極樂園
5        迪樂芮與泰迪這對在主題樂園西方極樂園中的人造人接待員情侶遭一名神秘的公園來客黑衣人攻擊  西方極樂園


In [10]:
df_train.label.value_counts() / len(df_train)
df_train.to_csv("train.tsv", sep="\t", index=False)
df_train.label.value_counts() / len(df_train)

獵魔士      0.146745
西方極樂園    0.139022
魷魚遊戲     0.130195
黑鏡       0.128724
怪奇物語     0.120265
絕命毒師     0.118058
黑袍糾察隊    0.113645
絕命律師     0.103347
Name: label, dtype: float64

In [11]:
df_val.to_csv("val.tsv", sep="\t", index=False)
df_val.label.value_counts() / len(df_val)

絕命律師     0.334633
黑鏡       0.187320
絕命毒師     0.176036
怪奇物語     0.106278
西方極樂園    0.084735
黑袍糾察隊    0.048215
獵魔士      0.046779
魷魚遊戲     0.016003
Name: label, dtype: float64

In [12]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_val.label.value_counts() / len(df_val)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index
            df_val = df_val[~df_val.index.isin(bad_label_index)]
    percentage=df_val.label.value_counts() / len(df_val)
print(percentage)


df_val.to_csv("val.tsv", sep="\t", index=False)
"""

'\npercentage=df_val.label.value_counts() / len(df_val)\nwhile max(percentage)>0.20:\n    for serie in series:\n        if percentage[serie]>0.20:\n            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index\n            df_val = df_val[~df_val.index.isin(bad_label_index)]\n    percentage=df_val.label.value_counts() / len(df_val)\nprint(percentage)\n\n\ndf_val.to_csv("val.tsv", sep="\t", index=False)\n'

In [13]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_total.label.value_counts() / len(df_total)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_total[df_total["label"]==serie].sample(len(df_total)//20).index
            df_total = df_total[~df_total.index.isin(bad_label_index)]
    percentage=df_total.label.value_counts() / len(df_total)
print(percentage)
"""
df_total.to_csv("total.tsv", sep="\t", index=False)

tokens_tensor：代表識別每個 token 的索引值，用 tokenizer 轉換即可  

segments_tensor：用來識別句子界限。第一句所有index對應 0，第二句所有index對應 1。另外注意句子間的 [SEP] 為 0  

masks_tensor：用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注  //因為我們會zero padding讓batch裡大小一致


In [14]:
label_map = {serie:i  for i, serie in enumerate(series)}
inv_label_map = {v: k for k, v in label_map.items()}
print(inv_label_map)


def indices_to_labels(indices):
    for i in range(len(indices)):
        indices[i] = inv_label_map[i]
    return indices


print(indices_to_labels([2, 3, 4]))

{0: '西方極樂園', 1: '絕命律師', 2: '怪奇物語', 3: '絕命毒師', 4: '獵魔士', 5: '黑鏡', 6: '魷魚遊戲', 7: '黑袍糾察隊'}
['西方極樂園', '絕命律師', '怪奇物語']


In [15]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset

#https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class SeriesDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer,label_map):
        #assert mode in ["train", "test","val"]  # 確定檔案名稱只接受train或test
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")   #讀處裡好的tsv避免跟所有檔案的csv搞混
        self.len = len(self.df)
        self.label_map = label_map
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer

    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        """
        if self.mode == "test":
            text, label = self.df.iloc[idx, :].values
            label_tensor = None
        else:
        """
        text,label = self.df.iloc[idx, :].values
        # 將 label 文字也轉換成索引方便轉換成 tensor
        label_id = self.label_map[label]
        label_tensor = torch.tensor(label_id)

        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        return (tokens_tensor, label_tensor)

    def __len__(self):
        return self.len

In [16]:
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = SeriesDataset("train", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
valset=SeriesDataset("val", tokenizer=tokenizer,label_map=label_map)
totalset=SeriesDataset("total", tokenizer=tokenizer,label_map=label_map)
#valset_replace = SeriesDataset("val_replace", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
print(trainset.df)
print(valset.df)
#print(valset_replace.df)

                                               sentence  label
0                                             就看有沒有第五季了  西方極樂園
1                        忍不住浮出來討論QQ但就千頭萬緒，沒辦法整理成有頭有尾的一篇  西方極樂園
2                                 前面幾季有著中央電腦在主導著人類社會的存在  西方極樂園
3                              直到德妹從人工園區進入人類社會並將中央電腦給解決  西方極樂園
4                              這一季德妹後來才知道自己成為了中央電腦的一段程式  西方極樂園
...                                                 ...    ...
2714  Amazon反英雄漫畫改編影集The Boys發佈預告Amazon 超暴力反英雄漫畫改編影集...  黑袍糾察隊
2715  影片中被眾人追捧的七大巨頭暫譯，The Seven外表光鮮亮麗，實際上卻目無法紀、暴力狂妄，...  黑袍糾察隊
2716     《星際爭霸戰》卡爾厄本飾黑袍糾察隊首領 Billy Butcher，是第一位加入該團隊的人物  黑袍糾察隊
2717                    Usher與《我是殭屍》奈森米契Nathan Mitchell  黑袍糾察隊
2718      《黑袍糾察隊》第一季共八集將於 2019 年夏季上線 Amazon Prime Video  黑袍糾察隊

[2719 rows x 2 columns]
                                               sentence  label
0                                         但是這世界下，各種暗流湧動  西方極樂園
1             部分機器人出現自我覺醒，發現了自己只是作為故事角色的存在，並且想擺脫樂園對其的控制  西方極樂園
2     樂園的管理層害怕樂園的創造者控制著樂園的一切而試

In [17]:
import random
# 選擇一個樣本
sample_idx = int(random.random()*len(trainset))
print(sample_idx)
# 將原始文本拿出做比較
text,label = trainset.df.iloc[sample_idx]

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor,label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

label_tensor   ：{label_tensor}
--------------------

[還原 tokens_tensors]
{combined_text}
""")

615
[原始文本]
句子 1：突然在S04E07開始的這一幕出現Saul 某人的名字
分類  ：絕命律師

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101,  4960,  4197,  1762,   161,  9099,  8154,  9131,  7274,  1993,
         4638,  6857,   671,  2391,  1139,  4412,  8945, 10086,  3378,   782,
         4638,  1399,  2099,   102])

label_tensor   ：1
--------------------

[還原 tokens_tensors]
[CLS]突然在s##04##e##07開始的這一幕出現sa##ul某人的名字[SEP]



In [18]:
print(type(trainset[0]))
print(len(trainset))
print(type(trainset))

#trainset 直接用[]來看是一個(2657,)個tuple of three tensor

<class 'tuple'>
2719
<class '__main__.SeriesDataset'>


In [19]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `MovieDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list由前面的3個tenors組成，裡頭的每個 element 都是
# 剛剛定義的 `MovieDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]       #把每個sample的tensor取出來放進list 產生一個 list of tensor (batch_size,)
    
    # 測試集有 labels
    if samples[0][1] is not None:
        label_ids = torch.stack([s[1] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度(要同時做好幾個RNN就要batch中每個元素一樣長)
    #https://blog.csdn.net/qq_43391414/article/details/123289492
    
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    
    return tokens_tensors, label_ids


In [20]:
#把data跟寫好的create_mini_batch放進trainloader
BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )

valloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
totalloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""
valloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""

'\nvalloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,\n                         shuffle=True,\n                         collate_fn=create_mini_batch\n                         )\n'

In [21]:
data = next(iter(trainloader))  # 取出traing loader 裡的一個mini_batch

tokens_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([32, 74]) 
tensor([[ 101, 6857, 2658,  ...,    0,    0,    0],
        [ 101,  852, 6857,  ...,    0,    0,    0],
        [ 101,  677, 4891,  ...,    0,    0,    0],
        ...,
        [ 101, 2523, 1914,  ...,    0,    0,    0],
        [ 101, 2207, 6303,  ...,    0,    0,    0],
        [ 101, 5424, 2715,  ...,  679, 3923,  102]])
------------------------
label_ids.shape        = torch.Size([32])
tensor([2, 3, 3, 0, 3, 2, 4, 0, 4, 0, 3, 1, 6, 4, 0, 0, 2, 3, 4, 6, 2, 4, 4, 6,
        0, 6, 7, 3, 0, 6, 3, 2])



建立 BERT 用的 mini-batch 時最需要注意的就是 zero padding 的存在了。  
你可以發現除了 lable_ids 以外，其他 3 個 tensors 的每個樣本的最後大都為 0，  
這是因為每個樣本的 tokens 序列基本上長度都會不同，需要補 padding。

In [22]:
from transformers import BertForSequenceClassification

In [23]:
import os
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"
NUM_LABELS = len(label_map)
PATH="model_series_sam100.pt"

model = BertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS,attention_probs_dropout_prob=0.10,
        hidden_dropout_prob=0.10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#uncommend to load model

if os.path.isfile(PATH):
    print("=====>LOAD checkpoint"+PATH)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("======>LOAD new model")


Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-c

======>LOAD new model


In [24]:
print(model.bert.embeddings.word_embeddings.weight.requires_grad)
model.bert.embeddings.word_embeddings.weight.requires_grad=False
print(model.bert.embeddings.word_embeddings.weight.requires_grad)

True
False


In [25]:
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=8, bias=True)


In [26]:
""""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        wrong_list=[]
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:     #檢查model是否在GPU
                data = [t.to("cuda:0") for t in data if t is not None]    #sentence 跟label tensor從tuple改放到cuda list
            tokens_tensors= data[0]   #data 已經放在cuda了 如果沒有的話 應該會報錯
            outputs = model(input_ids=tokens_tensors)
            #ouputs 原本是 SequenceClassifierOutput class
            #用outputs[0]可以取出他的tensor部分 shape=(64,3)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
 
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[1]
                total += labels.size(0)
                correct += (pred == labels).sum().item()   #.item把tensor 轉為純量
                
                cur=0
                for i in (pred!=labels):
                    if i==True:
                        wrong_list.append((tokens_tensors[cur],pred[cur].item(),labels[cur].item()))
                    cur+=1
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))  #最後會形成一長串所有的prediction
    
    if compute_acc:
        acc = correct / total
        return wrong_list, acc
    return predictions
    

In [27]:

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)     #把model移到devive 應該會是cuda:0
acc_record =0.5 #set 0 to start saving model
train_acc_list=[]
val_acc_list=[]

device: cuda:0


In [28]:
#for data in trainloader:
    #print(data[0].shape)
print(next(model.parameters()).shape)

torch.Size([21128, 768])


In [29]:
data = next(iter(trainloader))    #並沒有紀錄iter 所以不會一直next下去 只會取trainloader的開頭 並取得next
for t in data:
    print(t.shape)
tmp=[t.to("cuda:0") for t in data if t is not None]
print(type(tmp))
print(type(data))

torch.Size([32, 71])
torch.Size([32])
<class 'list'>
<class 'tuple'>


In [30]:
from sam import SAM

from utility.smooth_cross_entropy import smooth_crossentropy
from utility.initialize import initialize
from utility.step_lr import StepLR
from utility.bypass_bn import enable_running_stats, disable_running_stats

输入：
    input_ids：训练集，torch.LongTensor类型，shape是[batch_size, sequence_length]
    token_type_ids：可选项，当训练集是两句话时才有的。
    attention_mask：可选项，当使用mask才有，可参考原论文。
    labels：数据标签，torch.LongTensor类型，shape是[batch_size]

In [31]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來


EPOCHS = 15
FREQ=10
acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=tokens_tensors,
                        labels=labels)

        loss = outputs[0]
        logits=outputs[1]
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))
    PATH = "model_series_origin.pt"
    if val_acc > acc_record:
        acc_record = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, PATH)
        print("Save Checkpoint to:", PATH)


epoch=0 iteration=0 acc=0.03125
epoch=0 iteration=10 acc=0.0
epoch=0 iteration=20 acc=0.125
epoch=0 iteration=30 acc=0.21875
epoch=0 iteration=40 acc=0.15625
epoch=0 iteration=50 acc=0.28125
epoch=0 iteration=60 acc=0.34375
epoch=0 iteration=70 acc=0.21875
epoch=0 iteration=80 acc=0.5
[epoch 1] loss: 170.247, acc: 0.240,val acc: 0.233
epoch=1 iteration=0 acc=0.21875
epoch=1 iteration=10 acc=0.40625
epoch=1 iteration=20 acc=0.46875
epoch=1 iteration=30 acc=0.5
epoch=1 iteration=40 acc=0.46875
epoch=1 iteration=50 acc=0.5625
epoch=1 iteration=60 acc=0.375
epoch=1 iteration=70 acc=0.53125
epoch=1 iteration=80 acc=0.4375
[epoch 2] loss: 136.263, acc: 0.470,val acc: 0.268
epoch=2 iteration=0 acc=0.59375
epoch=2 iteration=10 acc=0.65625
epoch=2 iteration=20 acc=0.4375
epoch=2 iteration=30 acc=0.59375
epoch=2 iteration=40 acc=0.71875
epoch=2 iteration=50 acc=0.71875
epoch=2 iteration=60 acc=0.65625
epoch=2 iteration=70 acc=0.6875
epoch=2 iteration=80 acc=0.53125
[epoch 3] loss: 100.375, acc: 

In [32]:
%%time
#SAM to train
# 訓練模式
model.train()


#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.0003, momentum=0.9)
EPOCHS = 5
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
        train_acc_list.append(acc)
        val_acc_list.append(val_acc)
        print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
              (epoch + 1, running_loss, acc, val_acc))
        if val_acc > acc_record:
            acc_record = val_acc
            SAVE_PATH = "model_series_sam.pt"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, SAVE_PATH)
            print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.1875
epoch=0 iteration=10 acc=0.8125
epoch=0 iteration=20 acc=0.375
epoch=0 iteration=30 acc=0.96875
epoch=0 iteration=40 acc=0.75
epoch=0 iteration=50 acc=0.90625
epoch=0 iteration=60 acc=0.625
epoch=0 iteration=70 acc=1.0
epoch=0 iteration=80 acc=0.90625
[epoch 1] loss: 17.122, acc: 0.761,val acc: 0.498
epoch=1 iteration=0 acc=0.875
epoch=1 iteration=10 acc=0.90625
epoch=1 iteration=20 acc=0.90625
epoch=1 iteration=30 acc=0.90625
epoch=1 iteration=40 acc=0.9375
epoch=1 iteration=50 acc=0.90625
epoch=1 iteration=60 acc=1.0
epoch=1 iteration=70 acc=0.9375
epoch=1 iteration=80 acc=0.875
[epoch 2] loss: 15.442, acc: 0.878,val acc: 0.493
epoch=2 iteration=0 acc=0.90625
epoch=2 iteration=10 acc=0.96875
epoch=2 iteration=20 acc=0.90625
epoch=2 iteration=30 acc=0.90625
epoch=2 iteration=40 acc=1.0
epoch=2 iteration=50 acc=0.90625
epoch=2 iteration=60 acc=1.0
epoch=2 iteration=70 acc=0.90625
epoch=2 iteration=80 acc=0.875
[epoch 3] loss: 14.142, acc: 0.895,val acc: 0

In [33]:
PATH="model_series_sam.pt"
if os.path.isfile(PATH):
    print("=====>LOAD checkpoint"+PATH)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    print("to",device)

=====>LOAD checkpointmodel_series_sam.pt
to cuda:0


In [34]:
%%time
#SAM to Full data
# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.001, momentum=0.9)
EPOCHS = 12
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(totalloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))
    if val_acc > acc_record:
        acc_record = val_acc
        SAVE_PATH = "model_series_sam"+str(int(val_acc*100))+".pt"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, SAVE_PATH)
        print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.375
epoch=0 iteration=10 acc=0.125
epoch=0 iteration=20 acc=0.375
epoch=0 iteration=30 acc=0.25
epoch=0 iteration=40 acc=0.375
epoch=0 iteration=50 acc=0.625
epoch=0 iteration=60 acc=0.375
epoch=0 iteration=70 acc=0.5
epoch=0 iteration=80 acc=0.5
epoch=0 iteration=90 acc=0.125
epoch=0 iteration=100 acc=0.125
epoch=0 iteration=110 acc=0.25
epoch=0 iteration=120 acc=0.25
epoch=0 iteration=130 acc=0.5
epoch=0 iteration=140 acc=0.0
epoch=0 iteration=150 acc=0.125
epoch=0 iteration=160 acc=0.375
epoch=0 iteration=170 acc=0.25
epoch=0 iteration=180 acc=0.25
epoch=0 iteration=190 acc=0.25
epoch=0 iteration=200 acc=0.5
epoch=0 iteration=210 acc=0.125
epoch=0 iteration=220 acc=0.125
epoch=0 iteration=230 acc=0.25
epoch=0 iteration=240 acc=0.125
epoch=0 iteration=250 acc=0.125
epoch=0 iteration=260 acc=0.375
epoch=0 iteration=270 acc=0.375
epoch=0 iteration=280 acc=0.25
epoch=0 iteration=290 acc=0.375
epoch=0 iteration=300 acc=0.125
epoch=0 iteration=310 acc=0.625
epoch

epoch=4 iteration=90 acc=0.875
epoch=4 iteration=100 acc=0.5
epoch=4 iteration=110 acc=0.0
epoch=4 iteration=120 acc=0.375
epoch=4 iteration=130 acc=0.625
epoch=4 iteration=140 acc=0.25
epoch=4 iteration=150 acc=0.25
epoch=4 iteration=160 acc=0.125
epoch=4 iteration=170 acc=0.75
epoch=4 iteration=180 acc=0.875
epoch=4 iteration=190 acc=0.75
epoch=4 iteration=200 acc=0.5
epoch=4 iteration=210 acc=0.5
epoch=4 iteration=220 acc=0.375
epoch=4 iteration=230 acc=0.25
epoch=4 iteration=240 acc=0.375
epoch=4 iteration=250 acc=0.25
epoch=4 iteration=260 acc=0.625
epoch=4 iteration=270 acc=0.5
epoch=4 iteration=280 acc=0.625
epoch=4 iteration=290 acc=0.5
epoch=4 iteration=300 acc=0.375
epoch=4 iteration=310 acc=0.75
epoch=4 iteration=320 acc=0.5
epoch=4 iteration=330 acc=0.125
epoch=4 iteration=340 acc=0.25
epoch=4 iteration=350 acc=0.0
epoch=4 iteration=360 acc=0.75
epoch=4 iteration=370 acc=0.75
epoch=4 iteration=380 acc=0.125
epoch=4 iteration=390 acc=0.375
epoch=4 iteration=400 acc=0.875
epo

epoch=8 iteration=200 acc=0.5
epoch=8 iteration=210 acc=0.375
epoch=8 iteration=220 acc=0.5
epoch=8 iteration=230 acc=0.0
epoch=8 iteration=240 acc=0.5
epoch=8 iteration=250 acc=0.25
epoch=8 iteration=260 acc=0.625
epoch=8 iteration=270 acc=0.625
epoch=8 iteration=280 acc=0.125
epoch=8 iteration=290 acc=0.25
epoch=8 iteration=300 acc=0.375
epoch=8 iteration=310 acc=0.25
epoch=8 iteration=320 acc=0.0
epoch=8 iteration=330 acc=0.75
epoch=8 iteration=340 acc=0.625
epoch=8 iteration=350 acc=0.75
epoch=8 iteration=360 acc=0.625
epoch=8 iteration=370 acc=0.625
epoch=8 iteration=380 acc=0.625
epoch=8 iteration=390 acc=0.25
epoch=8 iteration=400 acc=0.5
epoch=8 iteration=410 acc=0.625
epoch=8 iteration=420 acc=0.25
epoch=8 iteration=430 acc=0.25
epoch=8 iteration=440 acc=0.625
epoch=8 iteration=450 acc=0.5
epoch=8 iteration=460 acc=0.625
epoch=8 iteration=470 acc=0.5
epoch=8 iteration=480 acc=0.5
epoch=8 iteration=490 acc=0.625
epoch=8 iteration=500 acc=0.625
epoch=8 iteration=510 acc=0.5
epoc

In [35]:
#print wrong predict of val
model.eval()
wrong_list, acc = get_predictions(model, valloader, compute_acc=True)
print(acc)
print(wrong_list[0])
for wrong in wrong_list:
    tokens = tokenizer.convert_ids_to_tokens(wrong[0])
    combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string
    combined_text=combined_text.replace("[PAD]","")
    print(combined_text)
    print("predict:{} true:{}".format(inv_label_map[wrong[1]],inv_label_map[wrong[2]]))

0.8713582273286828
(tensor([ 101, 1060,  782, 6901, 1168, 5397, 3418, 4372, 7526, 4638, 5401, 1164,
        1830, 5474, 4673, 1751, 7380, 6725, 2936, 2929,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0], device='cuda:0'), 7, 0)
[CLS]兩人遭到羅根率領的美利堅聯盟國陸軍捕捉[SEP]
predict:黑袍糾察隊 true:西方極樂園
[CLS]沙奇亞再三保證自己絕不會向其他人説出這個秘密，而且法律也規定調查人員只能就理賠事件作證，透露無關事情均屬違法，但這都不能説動米婭[SEP]
predict:絕命律師 true:黑鏡
[CLS]飯桌上，擺了兩人份的晚餐，吉姆非常自然地坐下來，正跟自己的飯伴説話[SEP]
predict:絕命律師 true:怪奇物語
[CLS]儘管芭維塔接受，但卡蘭特拒絕了，隨後發生了爭吵[SEP]
predict:黑鏡 true:獵魔士
[CLS]車後是克拉克駕駛的廂式貨車，兩輛車一前一後沿着公路高速行駛[SEP]
predict:絕命律師 true:黑鏡
[CLS]但克利福德願意給傑米一個機會，下次再犯就絕不姑息[SEP]
predict:絕命毒師 true:絕命律師
[CLS]眼看比利佔優，史蒂夫要被打得不省人事，麥克斯突然拿起桌面上的一支針管，插進比利的脖子，並警告他不許動自己的朋友[SEP]
predict:絕命律師 true:怪奇物語
[CLS]這可害苦了飛船上的人，洛瑞因為替娜內特求情，被變成了

[CLS]傑克可以聽到凱莉説話，並將凱莉的意思表達出來[SEP]
predict:絕命毒師 true:黑鏡
[CLS]貝嘉與沃特集團簽了保密協定，隨後在一個設備先進的地方等待分娩，遺憾的是，預產期未到時嬰兒就撕裂了子宮，貝嘉因失血過多死亡，嬰兒胎死腹中[SEP]
predict:絕命毒師 true:黑袍糾察隊
[CLS]另一方面，俊昊潛水遊至最近的無人島上，卻發現手機信號微弱而難以傳出錄影證據，最後被負責人帶領隊伍追上[SEP]
predict:黑鏡 true:魷魚遊戲
[CLS]吉姆被人清理一番後，便被告知他可以離開了[SEP]
predict:絕命律師 true:怪奇物語
[CLS]原來當時古斯和自己的好兄弟古斯開了炸雞店，在炸雞店裏他們同時賣毒品[SEP]
predict:絕命律師 true:絕命毒師
[CLS]於是在一次激烈的爭吵後，傑米選擇離開競選活動[SEP]
predict:絕命毒師 true:黑鏡
[CLS]他一度接受法蘭西斯聲音的人工智慧言語治療，但是最終放棄[SEP]
predict:黑鏡 true:西方極樂園
[CLS]布徹被貝卡拒絕之後，隱居到阿姨茱蒂家[SEP]
predict:黑鏡 true:黑袍糾察隊
[CLS]還撬開隱藏在油畫後的保險箱，對物品拍了照，連保險箱裡的文件都沒放過[SEP]
predict:黑鏡 true:絕命律師
[CLS]在j##ess##e的建議下，三人用廢車處理場的巨型磁鐵來損壞筆記本，結果辦法奏效，筆記本由於跌落而損壞[SEP]
predict:黑鏡 true:絕命毒師
[CLS]mar##ie前去安慰sky##ler，得知真相後姐妹倆決裂[SEP]
predict:黑鏡 true:絕命毒師
[CLS]在電影院，史帝夫和蘿蘋開始眩暈，到廁所將身體被打入的藥全部吐掉而恢復正常，史帝夫剛想對蘿蘋表白，但蘿蘋袒露自己喜歡的是女孩[SEP]
predict:黑鏡 true:怪奇物語
[CLS]鎮上的孩子們都熱衷於在遊戲機室裏用遊戲比拼，分數越高的人，名字便在排行榜靠的越前[SEP]
predict:黑鏡 true:怪奇物語
[CLS]從銀行出來，霍克特駕車前往指定地點[SEP]
predict:絕命律師 true:黑鏡
[CLS]喬納森因而對喬伊釋懷，還就偷拍舉動向南茜敞開心扉[SEP]
predict:黑鏡 true:

[CLS]能源部人員包圍起孩子們[SEP]
predict:黑鏡 true:怪奇物語
[CLS]休伊的父親勸說兒子簽署那份協定，畢竟對於他們家庭而言，也是筆不小的收入，但休伊卻說什麼不肯，轉身出了門[SEP]
predict:絕命律師 true:黑袍糾察隊
[CLS]會議結束後，他重新回顧一下記憶，仔細思考老闆的謊言[SEP]
predict:絕命律師 true:黑鏡
[CLS]他找到他原先的伴侶科哈娜，並想帶她去看彼端山谷的工程，但到那裡時一切已被填埋[SEP]
predict:黑鏡 true:西方極樂園
[CLS]steven監視mike卻被戲耍[SEP]
predict:黑鏡 true:絕命毒師
[CLS]他們要翻過高牆，按照自己的意願生活在一起[SEP]
predict:絕命律師 true:黑鏡
[CLS]古斯察覺到不對勁，最後沒有選擇開車[SEP]
predict:絕命律師 true:絕命毒師
[CLS]羅洛隨口講解着，一邊擦着腦門上的汗[SEP]
predict:絕命律師 true:黑鏡
[CLS]屆時，會有人跟蹤瑪格麗特到他們的見面地點[SEP]
predict:黑鏡 true:絕命律師
[CLS]訂的披薩送來了，戴利暫停遊戲，回到現實中去開門[SEP]
predict:魷魚遊戲 true:黑鏡
[CLS]為了查清他們是誰派來刺探，將領向史帝夫和蘿蘋身上打入吐真劑，兩人在神志不清的情況下袒露對彼此有好感[SEP]
predict:西方極樂園 true:怪奇物語
[CLS]年輕律師連恩·福斯威爾li##amfox##well接受了一項工作評估，感覺自己做得不好[SEP]
predict:絕命律師 true:黑鏡
[CLS]但由於格里戈里前來視察，哈普和喬絲只能帶走阿列克謝而走為上策[SEP]
predict:獵魔士 true:怪奇物語
[CLS]瓦爾達克派出的外星大甲蟲張牙舞爪的衝了過來，正要應戰時，戴利手中的通訊器響了起來[SEP]
predict:西方極樂園 true:黑鏡
[CLS]因長時間的晝伏夜出，在不見天日的生活中，那些從德國找來的工人們變得煩燥不安，其中以卡伊尤為激烈[SEP]
predict:黑鏡 true:絕命律師
[CLS]塞拉爾也趁此機會，大開殺戒，任何妨礙他收購提洛的人都被清除[SEP]
predict:絕命律師 true:西方極

[CLS]伊恩對她進行了折磨，並且殺害了她[SEP]
predict:獵魔士 true:黑鏡
[CLS]反正亂世之中，誰又會在意多死兩個人[SEP]
predict:絕命律師 true:西方極樂園
[CLS]以現行法律，沙奇亞可以向法院申請調查令，但那需要半個月的時間[SEP]
predict:絕命律師 true:黑鏡
[CLS]一行人直奔軍械店搶購除魔所需的大量武器，但也被在場買槍的傑森發現行蹤[SEP]
predict:絕命毒師 true:怪奇物語
[CLS]唯一入獄的事由，還是34年前大學畢業典禮後的酒駕[SEP]
predict:黑鏡 true:絕命律師
[CLS]在修伊與阿兵哥追殺心靈風暴時，他試圖讓布徹困在自己的過去中以制服布徹[SEP]
predict:西方極樂園 true:黑袍糾察隊
[CLS]兩人越走越近並發生了一夜情，但事後關朵琳接獲顧問警告，要求她在競選期間與傑米保持距離[SEP]
predict:絕命毒師 true:黑鏡
[CLS]傑米事後非常懊悔自己的失控，但這番大放厥詞在youtube上大受歡迎並讓瓦爾多熊獲得更多支持[SEP]
predict:絕命毒師 true:黑鏡
[CLS]布徹和星光劫持路過司機的汽車，將修伊送至醫院治療[SEP]
predict:黑鏡 true:黑袍糾察隊
[CLS]吉姆十分無奈，只好由着十一[SEP]
predict:絕命律師 true:怪奇物語
[CLS]一次行動中，卡萊布沒有遵守[UNK][UNK]客[UNK]應用禁止交談的要求，擅自撕下目標嘴上的膠帶，遞過去一瓶水，弗朗西斯手機上隨即收到消滅卡萊布的指示[SEP]
predict:黑鏡 true:西方極樂園
[CLS]最後他在冷凍室找到了妻子，悲痛欲絕[SEP]
predict:黑鏡 true:西方極樂園
[CLS]目前在這所建築內，還生還的人寥寥無幾，大夥兒通力合作，四處搜尋裝備武器，打算殺出一條血路[SEP]
predict:絕命律師 true:怪奇物語
[CLS]古斯到養老院找到的叔叔，回憶起了二十年前的往事[SEP]
predict:絕命律師 true:絕命毒師
[CLS]至於霍華德和拉洛的屍體在哪裡？就被埋在超級實驗室裡，麥克從霍華德身上拿走他的皮包、鞋子和戒指，要創造出霍華德在海灘因為人生失敗而自殺的假象[SEP]
predict:黑袍糾察隊 

[CLS]艾蓮娜因梅芙對37號航班見死不救而與她分開[SEP]
predict:西方極樂園 true:黑袍糾察隊
[CLS]羅洛堅信今天會有很多參觀者，尼什望望連人影都沒有的大道，只能淡淡的一笑[SEP]
predict:絕命律師 true:黑鏡
[CLS]南茜心裏一直放不下芭芭拉，抱怨大家都像沒事兒似的在狂歡，還埋怨喬納森害死了芭芭拉[SEP]
predict:黑鏡 true:怪奇物語
[CLS]埃曼特勞特揹着金屬探測器在郊外綠洲汽車旅館廣告品後面的灌木叢裏，挖到了一具屍體，晚上回去後，為了隱藏自己，他用共用電話報警了[SEP]
predict:黑鏡 true:絕命律師
[CLS]監視人員每天都會把病房裏發生的情況反饋回來，包括哪些人來探視，哪位醫生來檢查[SEP]
predict:黑鏡 true:絕命律師
[CLS]警察在蓋爾家調查現場，他們發現有一本實驗記錄本，便帶走了它[SEP]
predict:絕命律師 true:絕命毒師
[CLS]他在自己的閣樓中，將玻璃片放入絲綢中保存[SEP]
predict:絕命律師 true:黑鏡
[CLS]莊園內外還有數名端著ak##47的守衛，可以說易守難攻[SEP]
predict:西方極樂園 true:絕命律師
[CLS]麥克在運貨的過程中遭到了搶劫，最終他還是保住了這批貨，但是他的耳朵受了重傷[SEP]
predict:絕命律師 true:絕命毒師
[CLS]在全世界一片混亂時，舊金山精神病院卻是出奇的安靜[SEP]
predict:黑鏡 true:西方極樂園
[CLS]在逃跑時，一群充滿敵意，被稱為鬼國的原住民夾擊美利堅聯盟國陸軍[SEP]
predict:黑袍糾察隊 true:西方極樂園
[CLS]卡莉的這份[UNK]大禮[UNK]，十一欣然接受[SEP]
predict:黑鏡 true:怪奇物語
[CLS]原來，橋並沒有下榻，剛才施展幻術的女孩伸手擦了擦自己的鼻血[SEP]
predict:黑鏡 true:怪奇物語
[CLS]不自由毋寧死，娜內特選擇了反抗，她想擺脱這個死循環的命運[SEP]
predict:西方極樂園 true:黑鏡
[CLS]警察從邁克的眼神中察覺窗外有異樣，立即出外搜捕[SEP]
predict:絕命律師 true:怪奇物語
[CLS]果然，吉姆此時在洞穴中被藤蔓纏繞着，狀態十分不妙，大夥兒

In [36]:
#plot
#plt.xlim(0,2200 ) # 設定 x 軸座標範圍
import matplotlib.pyplot as plt
plt.ylim(0, 1) # 設定 y 軸座標範圍

plt.xlabel('epoch', fontsize="10") # 設定 x 軸標題內容及大小
plt.ylabel('Accuracy', fontsize="10") # 設定 y 軸標題內容及大小
plt.title('Accuracy in use replace data (dropout=0.1)(8 movie prediction)', fontsize="18") # 設定圖表標題內容及大小

plt.plot(range(len(train_acc_list)),train_acc_list,marker='o', color='blue',label="Training Accuracy")
plt.plot(range(len(val_acc_list)),val_acc_list,marker='o', color='red',label="Validation Accuracy")
#for x,y in zip(range(len(val_acc_list)),val_acc_list): 
#    plt.text(x, y+0.01, "({}, {:.3f})".format(x,y))

plt.legend()
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.savefig("Movie Accuracy use replace data (dropout=0.1).png",dpi=100)

plt.clf()

<Figure size 1500x1000 with 0 Axes>

In [39]:
model.eval()
test_text = "[CLS]有大蜘蛛的恐怖遊戲[SEP]"  # 盡量湊到6個字以上 ex:xx到底是甚麼
tokens = tokenizer.tokenize(test_text)
ids = tokenizer.convert_tokens_to_ids(tokens)

id_tensor = torch.tensor([ids]).to(device)
print(test_text)
print(tokens[:])
print(ids[:])
k = 8
with torch.no_grad():
    outputs = model(input_ids=id_tensor)
    prediction = outputs[0]
    probs, indices = torch.topk(torch.softmax(prediction, -1), k)
    probs = probs.tolist()
    indices = indices.tolist()
    probs = probs[0]
    indices = indices[0]
for i, (indice, p) in enumerate(zip(indices, probs), 1):
    label = inv_label_map[indice]
    print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]))

[CLS]有大蜘蛛的恐怖遊戲[SEP]
['[CLS]', '有', '大', '蜘', '蛛', '的', '恐', '怖', '遊', '戲', '[SEP]']
[101, 3300, 1920, 6055, 6033, 4638, 2607, 2587, 6879, 2783, 102]
Top 1 (59%)：魷魚遊戲
Top 2 (23%)：黑鏡
Top 3 ( 9%)：怪奇物語
Top 4 ( 3%)：西方極樂園
Top 5 ( 1%)：絕命律師
Top 6 ( 1%)：獵魔士
Top 7 ( 0%)：絕命毒師
Top 8 ( 0%)：黑袍糾察隊
